# Dashboard

> Settings dashboard layout components

In [ ]:
#| default_exp components.dashboard

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, Optional
from pathlib import Path
from fasthtml.common import *
from fasthtml.common import FT
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import (
    flex_display, gap, flex, items, justify
)
from cjm_fasthtml_tailwind.utilities.spacing import p, m
from cjm_fasthtml_tailwind.utilities.sizing import container, max_w, w, min_h
from cjm_fasthtml_tailwind.core.base import combine_classes

from cjm_fasthtml_settings.components.sidebar import create_sidebar_menu, create_oob_sidebar_menu
from cjm_fasthtml_settings.core.utils import (
    load_config,
    get_default_values_from_schema,
)
from cjm_fasthtml_settings.components.forms import create_settings_form_container
from cjm_fasthtml_settings.core.html_ids import HtmlIds

## Form Skeleton

In [ ]:
#| export
def create_form_skeleton(
    schema_id: str,  # The schema ID for the settings
    hx_get_url: str  # URL to fetch the actual form content
) -> Div:  # Div element with loading trigger
    """Create a loading skeleton for the settings form that loads asynchronously.
    
    This provides a placeholder that triggers an HTMX request to load the actual form,
    improving perceived performance for complex forms.
    """
    return Div(
        # Loading spinner centered (could add actual loading UI here)
        Div(
            cls=combine_classes(
                flex_display,
                items.center,
                justify.center,
            )
        ),
        # HTMX attributes for async loading
        hx_get=hx_get_url,
        hx_trigger="load",
        hx_swap="outerHTML",
        id=HtmlIds.SETTINGS_CONTENT,
        cls=combine_classes(flex(1), min_h(0))
    )

## Render Schema Settings

In [ ]:
#| export
def render_schema_settings_content(
    schema: Dict,  # JSON schema for the settings
    config_dir: Optional[Path] = None  # Config directory path
) -> FT:  # Settings form container
    """Render settings content for a schema-based configuration.
    
    Args:
        schema: The JSON schema to render
        config_dir: Directory where configs are stored
    """
    from cjm_fasthtml_settings import routes as settings_rt
    
    # Get the schema identifier
    schema_id = schema.get("name")

    # Load existing config or use defaults
    saved_config = load_config(schema_id, config_dir)
    default_values = get_default_values_from_schema(schema)
    values = {**default_values, **saved_config}

    return create_settings_form_container(
        schema=schema,
        values=values,
        post_url=settings_rt.save.to(id=schema_id),
        reset_url=settings_rt.reset.to(id=schema_id),
        use_alert_container=True
    )

In [ ]:
# Example: Render schema settings
from cjm_fasthtml_settings.core.config import get_app_config_schema

schema = get_app_config_schema(app_title="Test App", include_theme=False)
content = render_schema_settings_content(schema=schema)

content

```html
<div>
  <div id="alert-container"></div>
<form enctype="multipart/form-data" hx-post="/settings/save?id=general" hx-swap="innerHTML" hx-target="#settings-content">    <div class="card bg-base-100 shadow-lg w-full max-w-4xl">
      <div class="card-body">
        <h2 class="card-title">Application Configuration</h2>
        <div class="text-base-content text-sm mb-6">General application settings and configuration</div>
        <div class="font-semibold text-base-content mb-3">Required Fields</div>
        <div class="mb-4">
<label class="label mb-1"><span class="font-medium text-base-content">App Title</span><span class="text-error font-bold"> *</span></label>          <input type="text" name="app_title" value="Test App" placeholder="Default: Test App" required minlength="1" maxlength="100" id="field-app_title" class="input input-md w-full">
          <p class="text-sm text-base-content mt-1 mb-2">Title displayed in the navbar and browser tab</p>
        </div>
        <div class="mb-4">
<label class="label mb-1"><span class="font-medium text-base-content">Config Dir</span><span class="text-error font-bold"> *</span></label>          <input type="text" name="config_dir" value="configs" placeholder="Default: configs" required pattern='^[^&lt;&gt;:"\|?*]+$' id="field-config_dir" class="input input-md w-full validator">
          <p class="text-sm text-base-content mt-1 mb-2">Directory where configuration files are stored</p>
        </div>
<hr class="my-6">        <div class="font-semibold text-base-content mb-3">Optional Fields</div>
        <div class="mb-4">
<label class="label flex items-center cursor-pointer">            <input type="checkbox" name="auto_open_browser" checked id="field-auto_open_browser" class="toggle toggle-primary">
<span class="font-medium text-base-content ml-2">Auto Open Browser</span></label>          <p class="text-sm text-base-content mt-1 mb-2">Automatically open browser when starting the application</p>
        </div>
        <div class="mb-4">
<label class="label flex items-center cursor-pointer">            <input type="checkbox" name="debug_mode" id="field-debug_mode" class="toggle toggle-primary">
<span class="font-medium text-base-content ml-2">Debug Mode</span></label>          <p class="text-sm text-base-content mt-1 mb-2">Enable debug mode for development</p>
        </div>
        <div class="mb-4">
          <div class="w-full">
<label class="label mb-1"><span class="font-medium text-base-content">Max Upload Size Mb</span></label><span class="text-xs text-base-content ml-2">Min: 1 • Max: 10240</span>          </div>
          <input type="number" name="max_upload_size_mb" value="100" min="1" max="10240" step="any" id="field-max_upload_size_mb" class="input input-md w-full">
          <p class="text-sm text-base-content mt-1 mb-2">Maximum file upload size in megabytes</p>
        </div>
        <div class="mb-4">
<label class="label flex items-center cursor-pointer">            <input type="checkbox" name="reload_on_change" id="field-reload_on_change" class="toggle toggle-primary">
<span class="font-medium text-base-content ml-2">Reload On Change</span></label>          <p class="text-sm text-base-content mt-1 mb-2">Automatically reload the application when files change (development only)</p>
        </div>
        <div class="mb-4">
<label class="label mb-1"><span class="font-medium text-base-content">Server Host</span></label><select name="server_host" id="field-server_host" class="select select-md w-full"><option>Select an option...</option><option value="0.0.0.0" selected>0.0.0.0</option><option value="127.0.0.1">127.0.0.1</option><option value="localhost">localhost</option></select>          <p class="text-sm text-base-content mt-1 mb-2">Host address for the web server</p>
        </div>
        <div class="mb-4">
          <div class="w-full">
<label class="label mb-1"><span class="font-medium text-base-content">Server Port</span></label><span class="text-xs text-base-content ml-2">Min: 1024 • Max: 65535</span>          </div>
          <input type="number" name="server_port" value="5000" min="1024" max="65535" step="1" id="field-server_port" class="input input-md w-full">
          <p class="text-sm text-base-content mt-1 mb-2">Port number for the web server</p>
        </div>
        <div class="mb-4">
          <div class="w-full">
<label class="label mb-1"><span class="font-medium text-base-content">Session Timeout Minutes</span></label><span class="text-xs text-base-content ml-2">Min: 0 • Max: 1440</span>          </div>
          <input type="number" name="session_timeout_minutes" value="60" min="0" max="1440" step="1" id="field-session_timeout_minutes" class="input input-md w-full">
          <p class="text-sm text-base-content mt-1 mb-2">Session timeout in minutes (0 for no timeout)</p>
        </div>
      </div>
    </div>
    <div class="card-actions mt-6 border-base-300">
<button type="submit" class="btn btn-primary">Save Configuration</button><button type="button" hx-get="/settings/reset?id=general" hx-swap="innerHTML" hx-target="#settings-content" class="btn btn-ghost ml-2">Reset to Defaults</button>    </div>
</form></div>

```

## Settings Content (Full Layout)

In [ ]:
#| export
def settings_content(
    request,  # FastHTML request object
    schema: Dict,  # Schema to display
    schemas: Dict,  # All registered schemas for sidebar
    config_dir: Optional[Path] = None,  # Config directory
    menu_section_title: str = "Settings"  # Sidebar section title
) -> Div:  # Settings content layout
    """Return settings content with sidebar and form.
    
    Handles both full page loads and HTMX partial updates.
    
    Args:
        request: The request object (to check for HTMX)
        schema: The schema to display
        schemas: All available schemas for the sidebar
        config_dir: Config directory path
        menu_section_title: Title for the sidebar menu section
    """
    from cjm_fasthtml_settings import routes as settings_rt
    
    # Get the schema identifier
    schema_id = schema.get("name")

    # Return content with updated sidebar menu via OOB swap if HTMX request to content area
    if request.headers.get('HX-Request') and request.headers.get('HX-Target') == HtmlIds.SETTINGS_CONTENT:
        updated_menu = create_oob_sidebar_menu(
            schemas=schemas,
            active_schema=schema_id,
            config_dir=config_dir,
            menu_section_title=menu_section_title
        )
        return Div(
            render_schema_settings_content(schema, config_dir),
            updated_menu
        )

    # Return full settings layout with immediate sidebar and async form
    return Div(
        # Sidebar - loads immediately
        Div(
            create_sidebar_menu(
                schemas=schemas,
                active_schema=schema_id,
                config_dir=config_dir,
                include_wrapper=True,
                menu_section_title=menu_section_title
            ),
            id=HtmlIds.SETTINGS_SIDEBAR,
            cls=combine_classes(w(64), flex.shrink_0)
        ),

        # Form skeleton - loads asynchronously
        create_form_skeleton(
            schema_id=schema_id,
            hx_get_url=settings_rt.load_form.to(id=schema_id)
        ),

        cls=combine_classes(
            flex_display,
            gap(6),
            container,
            max_w._7xl,
            m.x.auto,
            p(6)
        )
    )

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()